In [235]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Conv2D
from keras.layers import LSTM, Reshape

In [236]:
train_data = "/home/piyush/Documents/IEMOCAP/test/small_train"
test_data = "/home/piyush/Documents/IEMOCAP/test/small_test"

In [238]:
model = Sequential()
# define CNN model
model.add(TimeDistributed(Conv2D(32, (3, 3), activation = 'relu'),input_shape = (32, 239,720,1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
model.add(TimeDistributed(Flatten()))
# # define LSTM model
# model.add(TimeDistributed(Reshape((1355584,))))
model.add(LSTM(128, return_sequences = False))
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(8, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_294 (TimeDi (None, 32, 237, 718, 32)  320       
_________________________________________________________________
time_distributed_295 (TimeDi (None, 32, 118, 359, 32)  0         
_________________________________________________________________
time_distributed_296 (TimeDi (None, 32, 1355584)       0         
_________________________________________________________________
lstm_77 (LSTM)               (None, 128)               694125056 
_________________________________________________________________
dense_101 (Dense)            (None, 8)                 1032      
Total params: 694,126,408
Trainable params: 694,126,408
Non-trainable params: 0
_________________________________________________________________
None


In [239]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data,
                                                    target_size=(239, 720),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    color_mode = 'grayscale')

validation_generator = test_datagen.flow_from_directory(test_data,
                                                        target_size=(239, 720),
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        color_mode='grayscale')

Found 7559 images belonging to 8 classes.
Found 4022 images belonging to 8 classes.


In [240]:
model.fit_generator(
        train_generator,
        steps_per_epoch=7559,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=4022)

Epoch 1/5


ValueError: Error when checking input: expected time_distributed_294_input to have 5 dimensions, but got array with shape (32, 239, 720, 1)